<a href="https://colab.research.google.com/github/arnabksarkar/LLMFinetuning/blob/dev/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import random

## Mount Gdrive to get the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the data

In [3]:
# prompt: Generate a list of files and their content from the Google drive data_bengali_wiki folder

import os

# Specify the folder path in your Google Drive
folder_path = '/content/drive/MyDrive/data_bengali_wiki'  # Replace with your actual folder path

# Function to list files and their content
def list_files_and_content(folder_path):
    try:
        file_list = []
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                try:
                    with open(file_path, 'r', encoding='utf-8') as file: # Handle potential encoding issues
                        content = file.read()
                        if 'Bangladesh' not in content:
                          file_list.append({'filename': filename, 'content': content})
                except UnicodeDecodeError:
                    print(f"Error decoding file: {filename}. Skipping.")
                except Exception as e:
                    print(f"Error reading file {filename}: {e}")
            else:
                print(f"Skipping directory: {filename}")
        return file_list

    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
        return None

# Call the function and print results
files_data = list_files_and_content(folder_path)

if files_data:
    for file_info in files_data[:10]:
        print(f"File: {file_info['filename']}")
        print(f"Content:\n{file_info['content'][:200]}...\n") # Print first 200 characters for brevity

File: Chess - Wikipedia.txt
Content:

 Chess is a board game for two players. It is sometimes called international chess or Western chess to distinguish it from related games such as xiangqi (Chinese chess) and shogi (Japanese chess).
 C...

File: Barefooted  Indian who left Calcutta to join Celtic - The Scotsman.txt
Content:
 If you have already registered If you have not signed up previously Tuesday 
					31 Dec
 Light rain Temp High
    7°c
 Low
    4°c
 Wind From
     South
 Speed
    15 mph
 Wednesday 
					1 Jan
 Hea...

File: Ministry of Information & Cultural Affairs (West Bengal) - Wikipedia.txt
Content:

 The Department of Information & Cultural Affairs of West Bengal, previously known as the Department of Information & Public Relations, is a Bengal government ministry.[1] It is a nodal ministry main...

File: IFA Shield - Wikipedia.txt
Content:

 The IFA Shield is an annual football competition organized by the Indian Football Association, the football governing body in the 

### Load Gemini to generate Input and Answer from each texts.

Load Gemini first. And we will use prompts to generate translation for each file content.

Get the API Key from the userdata


In [4]:

# Import the genAI library
import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

In [5]:
len(files_data)

205

Generate the translation and see how it looks.

In [6]:
# prompt: use gemini to translate the texts in the files_data list

# ... (Your existing code)

genai.configure(api_key=GOOGLE_API_KEY)

def translate_text(text, target_language="bangla"):
    try:
        model = genai.GenerativeModel("gemini-1.5-pro-001")
        response = model.generate_content(
            f"Translate the following text to {target_language}:\n\n{text}",
            generation_config = genai.GenerationConfig(
                                    max_output_tokens=1000,
                                    temperature=0.1,
                                )
        )
        return response.text
    except Exception as e:
        print(f"Error during translation: {e}")
        return None


translated_files = []
for file_info in files_data:
    translated_text = translate_text(file_info['content'])
    print(f"translation done for file {file_info['filename']}")
    if translated_text:
        translated_files.append({
            'filename': file_info['filename'],
            'original_text': file_info['content'],
            'translated_text': translated_text
        })
    else:
        print(f"Skipping translation for {file_info['filename']} due to error.")

print(f'translated file sizes {len(translated_files)}')
# # Now you have a list of translated files in the 'translated_files' variable
# # You can further process or save this data as needed.
# # Example: Print the translated text for each file
# for translated_file in translated_files:
#     print(f"Original Filename: {translated_file['filename']}")
#     print(f"Translated Text:\n{translated_file['translated_text'][:200]}...\n") # Print first 200 characters

translation done for file Chess - Wikipedia.txt
translation done for file Barefooted  Indian who left Calcutta to join Celtic - The Scotsman.txt
translation done for file Ministry of Information & Cultural Affairs (West Bengal) - Wikipedia.txt
translation done for file IFA Shield - Wikipedia.txt
translation done for file Kolkata Police - Wikipedia.txt
translation done for file Alipurduar district - Wikipedia.txt
translation done for file Bhadu (festival) - Wikipedia.txt
translation done for file List of Indian state birds - Wikipedia.txt
translation done for file Horse racing - Wikipedia.txt
translation done for file Hindustan Ambassador - Wikipedia.txt
translation done for file 1954 East Bengal Legislative Assembly election - Wikipedia.txt
translation done for file ESPN - Wikipedia.txt
translation done for file Nadia Raj - Wikipedia.txt
translation done for file Help:Referencing for beginners - Wikipedia.txt
translation done for file Government of West Bengal - Wikipedia.txt
translati

In [7]:
# prompt: Save the above translated_files list in a new directory in my google drive wiki_bengali_translate. Use the filename property as name of the file and content as the translated_text

import os

# Define the new directory path in your Google Drive
new_directory = '/content/drive/MyDrive/wiki_bengali_translate'

# Create the directory if it doesn't exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# Save each translated file
for translated_file in translated_files:
    file_path = os.path.join(new_directory, translated_file['filename'])
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(translated_file['translated_text'])
        print(f"Saved translated text to: {file_path}")
    except Exception as e:
        print(f"Error saving file {file_path}: {e}")

Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Chess - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Barefooted  Indian who left Calcutta to join Celtic - The Scotsman.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Ministry of Information & Cultural Affairs (West Bengal) - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/IFA Shield - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Kolkata Police - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Alipurduar district - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Bhadu (festival) - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/List of Indian state birds - Wikipedia.txt
Saved translated text to: /content/drive/MyDrive/wiki_bengali_translate/Horse racing - W

In [8]:
# prompt: please find the difference between the filenames between translated_file and files_data list

# Find the difference between filenames in translated_files and files_data

translated_filenames = set(file_info['filename'] for file_info in translated_files)
files_data_filenames = set(file_info['filename'] for file_info in files_data)

# Find filenames present in files_data but not in translated_files
missing_in_translated = files_data_filenames - translated_filenames
print(f"Files in 'files_data' but not in 'translated_files': {missing_in_translated}")

# Find filenames present in translated_files but not in files_data
missing_in_files_data = translated_filenames - files_data_filenames
print(f"Files in 'translated_files' but not in 'files_data': {missing_in_files_data}")

Files in 'files_data' but not in 'translated_files': {'Flare - Wikipedia.txt'}
Files in 'translated_files' but not in 'files_data': set()
